In [1]:
import torch
import torch.nn as nn

In [2]:
from bnelearn.strategy import NeuralNetStrategy
from bnelearn.bidder import Bidder
from bnelearn.mechanism import FirstPriceSealedBidAuction

In [3]:
from copy import deepcopy

In [4]:
hidden_layer = 10
batch_size = 10
u_lo = 0
u_hi = 10
n_players = 2
n_items = 1

epochs = 100

In [5]:
model = NeuralNetStrategy(2, hidden_layer).cuda()

In [6]:
mechanism = FirstPriceSealedBidAuction(cuda=True)

In [7]:
optimizer = torch.optim.SGD(model.parameters(), lr=.01)

In [8]:
p1 =Bidder.uniform(u_lo,u_hi, model, batch_size = batch_size, n_players=2)

In [9]:
p2 = Bidder.uniform(u_lo,u_hi, model, batch_size = batch_size, n_players=2)

In [10]:
p1.draw_valuations_(), p2.draw_valuations_();

In [11]:
b1 = p1.get_action()
b2 = p2.get_action()

In [12]:
for epoch in range(epochs):
    # draw new valuations and bids
    p1.draw_valuations_(), p2.draw_valuations_()    
    b1 = p1.get_action()
    b2 = p2.get_action()    
    bids = torch.cat((b1,b2), 1).view(batch_size, n_players, 1)
    
    # run the auction and calculate utilities
    alloc, pay = mechanism.run(bids)
    
    u1 = p1.get_utility(alloc[:,0,:], pay[:,0]).mean()
    u2 = p2.get_utility(alloc[:,1,:], pay[:,1]).mean()

    loss = -u2
    
    if epoch % 50 == 0:
        print('epoch ', epoch, 'avg bid', (b1/p1.valuations).mean().item(), (b2/p2.valuations).mean().item(), 'utility', u1.item(), u2.item() )
    
    # propagate loss and update parameters   
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

epoch  0 avg bid 0.08133896440267563 0.10861232131719589 utility 2.9149539470672607 1.2897067070007324
epoch  50 avg bid 0.0007957854541018605 0.0068199606612324715 utility 3.4529941082000732 1.0677425861358643


In [13]:
alloc.shape

torch.Size([10, 2, 1])

# Same with ES

In [12]:
from collections import Iterable, deque
import torch
from torch.optim.optimizer import Optimizer, required
from copy import deepcopy

In [13]:
import torch.nn.utils as ut

In [14]:
from bnelearn.optimizer import ES
from bnelearn.environment import Environment

In [15]:
u_lo =0
u_hi =10

In [16]:
def strat_to_bidder(strategy, batch_size):
    return Bidder.uniform(u_lo,u_hi, strategy, batch_size = batch_size, n_players=2)

In [17]:
model = NeuralNetStrategy(2, hidden_layer).cuda()

In [18]:
env = Environment(mechanism, [], max_env_size=10, batch_size=512,
                  n_players=2, strategy_to_bidder_closure=strat_to_bidder)

In [19]:
es = ES(model=model, environment=env, lr=3e-4, sigma=.01, n_perturbations= 128)

In [22]:
epoch=100

In [ ]:
for e in range(epoch):
    print("utility: {}".format(-es.step().item()))
    

0.3553626835346222
utility: 3.3132572174072266
0.36968204379081726
utility: 3.358612537384033
0.3025469481945038
utility: 3.284991502761841
0.5179157853126526
utility: 3.6470067501068115
0.2819386422634125
utility: 3.478255033493042
0.54628586769104
utility: 3.357280731201172
0.7478052377700806
utility: 3.229257345199585
0.471486896276474
utility: 3.24820876121521
0.40410032868385315
utility: 3.2743842601776123
0.3575887084007263
utility: 3.313528537750244
0.4180537164211273
utility: 3.3347766399383545
0.35805538296699524
utility: 3.4002556800842285
0.3928188979625702
utility: 3.3228821754455566
0.3370993435382843
utility: 3.2984511852264404
0.3922747075557709
utility: 3.3841934204101562
0.4463036358356476
utility: 3.3408455848693848
0.3854764401912689
utility: 3.3080170154571533
0.41821226477622986
utility: 3.385808229446411
0.3643462359905243
utility: 3.2068002223968506
0.3445335328578949
utility: 3.333181381225586
0.3957439064979553
utility: 3.3440518379211426
0.43236109614372253
ut